In [3]:
import cloudscraper
from bs4 import BeautifulSoup as bs
import webbrowser
import time
import random
import json
import os
# 網頁 URL
url = "https://www.mobile01.com/topiclist.php?f=802"

# 使用 cloudscraper 來繞過 Cloudflare 的挑戰
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}
scraper = cloudscraper.create_scraper()
res = scraper.get(url, headers=headers).text

# 使用 BeautifulSoup 解析 HTML
soup = bs(res, "lxml")
# 找到所有分頁按鈕並提取最後一個頁碼
pagination_elements = soup.select('li.l-pagination__page > a.c-pagination[data-page]')
if not pagination_elements:
    raise ValueError("未找到任何分頁按鈕，請檢查選擇器是否正確。")

# 提取最後一個分頁的 data-page 值
last_page_number = int(pagination_elements[-1]["data-page"])
print("最後一個頁碼是:", last_page_number)

最後一個頁碼是: 30


In [5]:
def extract_article_info(url):
    res = scraper.get(url, headers=headers)

    # 測試關鍵字是否存在於 HTML 內（代表 Cloudflare 沒擋）
    if "l-articlePage__publish" not in res.text:
        print(f"⚠️ 疑似被 Cloudflare 擋下，內容如下（前300字）：\n{res.text[:300]}")
        skipped_links.append(url)
        return None


In [6]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time

# 建立強化版 scraper
scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'chrome',
        'platform': 'windows',
        'mobile': False
    }
)

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "zh-TW,zh;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://www.google.com"
}

# 目標頁面列表
base_url = "https://www.mobile01.com/"
pages = [f"https://www.mobile01.com/topiclist.php?f=802&p={i}" for i in range(1, 100)]

article_links = []
skipped_links = []

# 收集主文連結（排除回文）
for page in pages:
    res = scraper.get(page, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    links = soup.select('a.c-link.u-ellipsis[href*="topicdetail.php?f=802"]')
    for link in links:
        href = link.get("href").split("#")[0]
        if href and "topicdetail.php" in href and "&p=" not in href:
            full_url = base_url + href
            if full_url not in article_links:
                article_links.append(full_url)

print(f"✅ 共抓到 {len(article_links)} 筆主文連結")

# 抓取主文內容
def extract_article_info(url):
    res = scraper.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    publish_block = soup.select_one(".l-publish")
    content_block = soup.select_one('div[itemprop="articleBody"]')

    if not publish_block or not content_block:
        print(f"❌ 頁面缺少主文區塊或內文：{url}")
        skipped_links.append(url)
        return None

    title = soup.select_one("h1.t2")  # 有些情況 title 不在 publish_block 裡
    time_el = soup.select_one(".l-navigation__item .o-fNotes")
    tags = soup.select(".l-input__item a.o-hashtag")

    result = {
        "url": url,
        "title": title.text.strip() if title else "",
        "post_time": time_el.text.strip() if time_el else "",
        "content": content_block.text.strip() if content_block else "",
        "tags": ", ".join([tag.text.strip() for tag in tags]) if tags else ""
    }

    return result

# 執行爬取
results = []
for idx, link in enumerate(article_links):
    print(f"🔍 正在處理第 {idx+1} 筆：{link}")
    try:
        info = extract_article_info(link)
        if info:
            results.append(info)
    except Exception as e:
        print(f"[錯誤] {link} 發生異常：{e}")
    time.sleep(1)

# 匯出為 CSV
df = pd.DataFrame(results)
df.to_csv("mobile01_articles802_fixed.csv", index=False, encoding='utf-8-sig')
print(f"\n✅ 已成功輸出 {len(df)} 筆主文資料至 mobile01_articles802_fixed.csv")

# 匯出錯誤連結
if skipped_links:
    with open("skipped_links_802_fixed.txt", "w", encoding="utf-8") as f:
        for link in skipped_links:
            f.write(link + "\n")
    print(f"⚠️ 有 {len(skipped_links)} 筆主文無法擷取，已記錄至 skipped_links_802_fixed.txt")
else:
    print("🎉 所有主文皆成功擷取！")


✅ 共抓到 180 筆主文連結
🔍 正在處理第 1 筆：https://www.mobile01.com/topicdetail.php?f=802&t=6757649
🔍 正在處理第 2 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7104102
🔍 正在處理第 3 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7101269
🔍 正在處理第 4 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7095792
🔍 正在處理第 5 筆：https://www.mobile01.com/topicdetail.php?f=802&t=6673343
🔍 正在處理第 6 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7056008
🔍 正在處理第 7 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7072558
🔍 正在處理第 8 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7071324
🔍 正在處理第 9 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7045907
🔍 正在處理第 10 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7088230
🔍 正在處理第 11 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7055898
🔍 正在處理第 12 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7081782
🔍 正在處理第 13 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7041530
🔍 正在處理第 14 筆：https://www.mobile01.com/topicdetail.php?f=802&t=7075292
🔍 正在處理第 15 筆：

In [2]:

import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# 建立強化版 scraper
scraper = cloudscraper.create_scraper(browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False})

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "zh-TW,zh;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://www.google.com"
}

# ---------- 工具函式 ----------

def get_total_pages(url):
    """取得某主文的總頁數"""
    res = scraper.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    pagination_links = soup.select("a.c-pagination")
    pages = [int(re.search(r'p=(\d+)', a.get('href', '')).group(1)) for a in pagination_links if re.search(r'p=(\d+)', a.get('href', ''))]
    return max(pages) if pages else 1

def extract_article_pages(base_url):
    """抓取單一主文所有分頁樓層"""
    all_floors = []
    total_pages = get_total_pages(base_url)

    for page in range(1, total_pages + 1):
        full_url = f"{base_url}&p={page}" if "?p=" not in base_url else base_url
        try:
            print(f"📄 抓取：{full_url}")
            res = scraper.get(full_url, headers=headers)
            soup = BeautifulSoup(res.text, "html.parser")

            title = soup.select_one("h1.t2")
            floor_blocks = soup.select('div[itemprop="articleBody"], article.c-articleLimit')
            time_tags = soup.select('span.o-fNotes.o-fSubMini')

            for i, block in enumerate(floor_blocks):
                content = block.get_text(strip=True)
                time_text = time_tags[i].text.strip() if i < len(time_tags) else ""
                all_floors.append({
                    "url": base_url,
                    "page": page,
                    "title": title.text.strip() if title else "",
                    "post_time": time_text,
                    "content": content
                })

            time.sleep(1)
        except Exception as e:
            print(f"❌ 抓取錯誤：{full_url} | {e}")
            break

    return all_floors

# ---------- 主文連結抓取 ----------

base_url = "https://www.mobile01.com/"
pages = [f"https://www.mobile01.com/topiclist.php?f=802&p={i}" for i in range(1, 100)]

article_links = []
for page in pages:
    res = scraper.get(page, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    links = soup.select('a.c-link.u-ellipsis[href*="topicdetail.php?f=802"]')
    for link in links:
        href = link.get("href").split("#")[0]
        if href and "topicdetail.php" in href and "&p=" not in href:
            full_url = base_url + href
            if full_url not in article_links:
                article_links.append(full_url)

print(f"\n✅ 共抓到 {len(article_links)} 筆主文連結")

# ---------- 抓取每篇文章所有樓層 ----------
all_results = []
skipped_links = []

for idx, link in enumerate(article_links):
    print(f"\n🔍 正在處理第 {idx+1} 筆：{link}")
    try:
        floors = extract_article_pages(link)
        if floors:
            all_results.extend(floors)
        else:
            skipped_links.append(link)
    except Exception as e:
        print(f"[錯誤] {link} 發生異常：{e}")
        skipped_links.append(link)

# ---------- 匯出結果 ----------
df = pd.DataFrame(all_results)
df.to_csv("mobile01_full_articles_802.csv", index=False, encoding='utf-8-sig')
print(f"\n✅ 已成功輸出 {len(df)} 筆樓層資料至 mobile01_full_articles_802.csv")

if skipped_links:
    with open("skipped_links_802.txt", "w", encoding="utf-8") as f:
        for link in skipped_links:
            f.write(link + "\n")
    print(f"⚠️ 有 {len(skipped_links)} 筆主文未成功處理，已記錄至 skipped_links_802.txt")
else:
    print("🎉 所有主文皆成功擷取！")



✅ 共抓到 180 筆主文連結

🔍 正在處理第 1 筆：https://www.mobile01.com/topicdetail.php?f=802&t=6757649
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=1
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=2
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=3
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=4
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=5
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=6
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=7
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=8
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=9
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=10
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=11
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=12
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802&t=6757649&p=13
📄 抓取：https://www.mobile01.com/topicdetail.php?f=802